In [300]:
import numpy as np

import sys
print(sys.getrecursionlimit())
sys.setrecursionlimit(10000)

10000


In [301]:
class Day20():

    def __init__(self, data_link):
        self.data_flat = open(data_link, 'r').read().split('\n')[:-1]
        self.board = np.array([[elem for elem in line] for line in self.data_flat], dtype = '<U8')
        self.start = np.argwhere(self.board == 'S')[0]
        self.end = np.argwhere(self.board == 'E')[0]
        self.directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]

    def __str__(self):
        return '\n'.join([''.join(list(line)) for line in self.board])

    def sum_tuple(self, tuple_a, tuple_b):
        return tuple([tuple_a[index] + tuple_b[index] for index in range(len(tuple_a))]) if len(tuple_a)==len(tuple_b) else False

    def get_neighbours(self, pos):
        if pos[0] in [0, len(self.board)-1] or pos[1] in [0, len(self.board)-1]:
            return []
        return [self.sum_tuple(pos, direction) for direction in self.directions if self.board[self.sum_tuple(pos, direction)] in ['E', '.']]

    def solve(self, pos = None, dist = 0):
        if not pos:
            pos = tuple(self.start)
        if self.board[pos] == 'E':
            self.board[pos] = 0
            return 0
        self.board[pos] = 'x'
        for neigh in self.get_neighbours(pos):
            dist = 1 + self.solve(neigh)
        self.board[pos] = dist
        return dist

    def get_walls(self):
        return [tuple(wall) for wall in np.argwhere(self.board == '#') if len(self.get_neighbours(wall)) >= 2]

    def solve_cheating(self, pos_wall):
        neighs = [int(self.board[self.sum_tuple(pos_wall, direction)]) for direction in self.directions if (self.board[self.sum_tuple(pos_wall, direction)]).isnumeric()]
        return max(neighs)-min(neighs)-2

    def main(self):
        self.board[tuple(self.start)] = '.'
        walls = self.get_walls()
        self.solve()

        # -> To get all cheats :
        # result = {}
        # for wall in walls:
        #     result[self.solve_cheating(wall)] = result.get(self.solve_cheating(wall), 0) +1
        #     if self.solve_cheating(wall) == 4:
        #         print(wall)
        # return result

        return len([wall for wall in walls if self.solve_cheating(wall) >= 100])

    def get_far_neighs(self, pos, dist = 20):
        result = []
        x, y = pos
        for d in range(dist+1):
            for a in range(0, d+1):
                if x+a < len(self.board) and y+d-a < len(self.board[0]):
                    result.append((x+a, y+d-a))
                if x-a >= 0 and y+d-a < len(self.board[0]):
                    result.append((x-a, y+d-a))
                if x+a < len(self.board) and y+a-d >= 0:
                    result.append((x+a, y+a-d))
                if x-a >= 0 and y+a-d >= 0:
                    result.append((x-a, y+a-d))
        return [elem for elem in list(set(result)) if self.board[elem].isnumeric()]

    def get_dist_cheated(self, pos):
        far_neighs = self.get_far_neighs(pos)
        result = []
        for neigh in far_neighs:
            dist = abs(pos[0] - neigh[0]) + abs(pos[1] - neigh[1])
            if int(self.board[pos]) - dist > int(self.board[neigh]):
                result.append(int(self.board[pos]) - int(self.board[neigh]) - dist)
        return result

    def main_2(self, dist_max = 100):
        self.board[tuple(self.start)] = '.'
        positions = np.argwhere(self.board == '.')
        self.solve()
        result = {}
        total = 0
        for elem in positions:
            cheats = self.get_dist_cheated(tuple(elem))
            for cheat in cheats:
                result[cheat] = result.get(cheat, 0) +1
                if cheat >= dist_max:
                    total += 1
        return total


In [302]:
day20_test = Day20('other/day20_test.txt')
print(day20_test)

###############
#...#...#.....#
#.#.#.#.#.###.#
#S#...#.#.#...#
#######.#.#.###
#######.#.#...#
#######.#.###.#
###..E#...#...#
###.#######.###
#...###...#...#
#.#####.#.###.#
#.#...#.#.#...#
#.#.#.#.#.#.###
#...#...#...###
###############


In [303]:
day20 = Day20('data/day20.txt')
day20.main()

1317

In [304]:
day20_test = Day20('other/day20_test.txt')
day20_test.main_2(50)

285

In [305]:
day20 = Day20('data/day20.txt')
day20.main_2()

982474